In [1]:
# This is the URL for the supplementary table
url = 'https://www.cell.com/cms/10.1016/j.cell.2023.07.013/attachment/b342834f-0ab7-4d68-be07-66e9ba38e3df/mmc3.xlsx'

In [2]:
import pandas as pd

In [3]:
# Naively trying to load the table from the URL errors with 403: Forbidden
# sheets = pd.read_excel(url)

In [4]:
# Load a local copy of the table and select Table 3G
df = pd.read_excel('mmc3.xlsx', sheet_name='Table 3G')
df

,Unnamed: 0,gene_name,logFC,AveExpr,t,P.Value,adj.P.Val,B,qval,propMissing,...,propMissingOut,id,id.description,variableSites,accession_number,feature,gsea_rank,gsea_rank_p,causalpath_adjusted_id,prot_residue
0,NP_001269315.1_K345k_1_1_345_345,IDH1,-3.904846,-0.941172,-6.944246,8.017521e-07,0.000565,4.015043,0.000440,0.685185,...,0.583333,HRD,isocitrate dehydrogenase [NADP] cytoplasmic G...,"['K345k', 'K345k']",NP_001269315.1,acetylome,-7.854981e+00,-23.803785,NP_001269315.1_K345k_1_1_345_345,IDH1_K345k_1_1_345_345
1,NP_006752.1_K142k_1_1_142_142,YWHAE,-1.019937,0.148372,-4.070577,1.611649e-04,0.049597,0.820275,0.038650,0.111111,...,0.125000,HRD,14-3-3 protein epsilon GN=YWHAE,"['K142k', 'K142k']",NP_006752.1,acetylome,-1.209124e+00,-3.868344,NP_006752.1_K142k_1_1_142_142,YWHAE_K142k_1_1_142_142
2,NP_001609.2_K105k_1_1_105_105,PARP1,1.236072,-0.516186,3.981427,2.110521e-04,0.049597,0.579085,0.038650,0.092593,...,0.083333,HRD,poly [ADP-ribose] polymerase 1 GN=PARP1,"['K105k', 'K105k']",NP_001609.2,acetylome,1.428431e+00,4.543317,NP_001609.2_K105k_1_1_105_105,PARP1_K105k_1_1_105_105
3,NP_001122321.1_K455k_1_1_455_455,SMARCA4,0.913719,-0.796626,3.485168,9.466467e-04,0.118682,-0.768856,0.092486,0.000000,...,0.000000,HRD,transcription activator BRG1 isoform A GN=SMA...,"['K455k', 'K455k']",NP_001122321.1,acetylome,8.427154e-01,2.762915,NP_001122321.1_K455k_1_1_455_455,SMARCA4_K455k_1_1_455_455
4,NP_001609.2_K621k_1_1_621_621,PARP1,0.734708,-0.190376,3.490196,9.621225e-04,0.118682,-0.770646,0.092486,0.055556,...,0.041667,HRD,poly [ADP-ribose] polymerase 1 GN=PARP1,"['K621k', 'K621k']",NP_001609.2,acetylome,6.776145e-01,2.216444,NP_001609.2_K621k_1_1_621_621,PARP1_K621k_1_1_621_621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5388,ENSG00000097007.19,ABL1,0.002431,5.878027,0.017314,9.862142e-01,0.993643,-6.474316,0.406492,NaN,...,NaN,HRD,NaN,NaN,NaN,transcriptome,5.029325e-06,0.000015,ENSG00000097007.19,ENSG00000097007.19
5389,ENSG00000102977.17,ACD,-0.001802,2.935979,-0.012782,9.898221e-01,0.994550,-6.062690,0.406863,NaN,...,NaN,HRD,NaN,NaN,NaN,transcriptome,-2.292426e-06,-0.000008,ENSG00000102977.17,ENSG00000102977.17
5390,ENSG00000167325.15,RRM1,-0.001353,6.557619,-0.011514,9.908319e-01,0.994550,-6.503115,0.406863,NaN,...,NaN,HRD,NaN,NaN,NaN,transcriptome,-1.463072e-06,-0.000005,ENSG00000167325.15,ENSG00000167325.15
5391,ENSG00000161036.13,LRWD1,-0.001006,3.850618,-0.007013,9.944159e-01,0.996278,-6.224021,0.407570,NaN,...,NaN,HRD,NaN,NaN,NaN,transcriptome,-7.332363e-07,-0.000002,ENSG00000161036.13,ENSG00000161036.13


In [5]:
# Filter the table to adjusted p-values less than 0.055 to retain significant results
df = df[df['adj.P.Val'] < 0.055]

In [6]:
# Look at statistics of different modification types that are significant
from collections import Counter
Counter(df.feature)

Counter({'phosphoproteome': 206,
         'transcriptome': 124,
         'phosphoproteome_res': 69,
         'proteome': 64,
         'acetylome': 3,
         'acetylome_res': 1})

In [7]:
# Construct a list of all phosphorylation sites that have significantly
# increased compared to control, and represent these as tuples compatible
# with Protmapper (gene_name, 'hgnc', residue, position).
import re
sites = set()
for _, row in df.iterrows():
    if row['feature'] == 'phosphoproteome' and row['logFC'] > 0:
        matches = re.findall(r'([STY]\d+)', row['variableSites'])
        sites |= {(row['gene_name'], 'hgnc', match[0], match[1:]) for match in matches}
sites

{('ATAD5', 'hgnc', 'S', '44'),
 ('ATR', 'hgnc', 'T', '1989'),
 ('ATRIP', 'hgnc', 'S', '224'),
 ('ATRIP', 'hgnc', 'S', '239'),
 ('ATRIP', 'hgnc', 'S', '518'),
 ('BAZ1B', 'hgnc', 'S', '330'),
 ('BAZ1B', 'hgnc', 'S', '349'),
 ('BLM', 'hgnc', 'S', '28'),
 ('BOD1L1', 'hgnc', 'S', '2905'),
 ('BRCA2', 'hgnc', 'S', '93'),
 ('BRIP1', 'hgnc', 'S', '226'),
 ('CDC25B', 'hgnc', 'S', '321'),
 ('CDC25B', 'hgnc', 'S', '353'),
 ('CDC25B', 'hgnc', 'S', '375'),
 ('CDC6', 'hgnc', 'S', '54'),
 ('CDK1', 'hgnc', 'T', '14'),
 ('CDK1', 'hgnc', 'T', '161'),
 ('CDK1', 'hgnc', 'Y', '15'),
 ('CHAF1A', 'hgnc', 'S', '775'),
 ('CHEK2', 'hgnc', 'S', '303'),
 ('CHTF18', 'hgnc', 'S', '225'),
 ('CHTF18', 'hgnc', 'S', '64'),
 ('CHTF18', 'hgnc', 'S', '871'),
 ('CLSPN', 'hgnc', 'S', '225'),
 ('CLSPN', 'hgnc', 'S', '83'),
 ('CLSPN', 'hgnc', 'S', '846'),
 ('CLSPN', 'hgnc', 'T', '1287'),
 ('CUL4B', 'hgnc', 'S', '180'),
 ('DBF4', 'hgnc', 'S', '359'),
 ('DBF4', 'hgnc', 'S', '381'),
 ('DBF4', 'hgnc', 'S', '508'),
 ('DBF4', 'hgnc'

In [8]:
# Use the protmapper to map the sites to human reference
import protmapper
mapped_sites = protmapper.default_mapper.map_sitelist_to_human_ref(sites)

Mapping sites: 0it [00:00, ?it/s]INFO: [2024-06-25 12:55:01] protmapper.uniprot_client - Loading Swissprot sequences...
INFO: [2024-06-25 12:55:03] protmapper.uniprot_client - Loading Uniprot isoform sequences...
Mapping sites: 189it [00:11, 16.96it/s]


In [10]:
# Print counts of mapping results
Counter([ms.description for ms in mapped_sites])

Counter({'VALID': 174, 'NO_MAPPING_FOUND': 15})

In [11]:
# Filter to valid or validly mapped sites
valid_sites = [ms for ms in mapped_sites if ms.valid or ms.mapped_id]

In [15]:
# Query the INDRA DB for Phosphorylation statements whose substrate is one
# of the proteins whose phosphorylation appears in the site list
from indra.sources.indra_db_rest import get_statements_from_query
from indra.sources.indra_db_rest.query import HasAgent, HasType

In [16]:
stmts_by_target = {}
unique_genes = {ms.gene_name for ms in valid_sites}

In [18]:
import tqdm
for gene in tqdm.tqdm(unique_genes):
    q = HasAgent(gene, role='OBJECT') & HasType('Phosphorylation')
    ip = get_statements_from_query(q)
    stmts_by_target[gene] = ip.statements

  0%|                                                   | 0/78 [00:00<?, ?it/s]INFO: [2024-06-25 12:57:00] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=CHTF18 with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:57:00] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:57:00] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:57:00] indra_db_rest.request_logs -   OFFSET: 0
  1%|▌                                          | 1/78 [00:01<01:39,  1.29s/it]INFO: [2024-06-25 12:57:02] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=RIF1 with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:57:02] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:57:02] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:57:02] indra_db_rest.request_logs -   OFFSET: 0
  3%|█                                        

INFO: [2024-06-25 12:57:36] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:57:36] indra_db_rest.request_logs -   OFFSET: 0
 22%|█████████▏                                | 17/78 [00:36<01:58,  1.95s/it]INFO: [2024-06-25 12:57:37] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=POLR2C with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:57:37] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:57:37] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:57:37] indra_db_rest.request_logs -   OFFSET: 0
 23%|█████████▋                                | 18/78 [00:37<01:37,  1.63s/it]INFO: [2024-06-25 12:57:37] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=RECQL5 with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:57:37] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:57:37] indra_db_rest.re

 44%|██████████████████▎                       | 34/78 [01:08<01:17,  1.76s/it]INFO: [2024-06-25 12:58:08] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=SMARCC1 with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:58:08] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:58:08] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:58:08] indra_db_rest.request_logs -   OFFSET: 0
 45%|██████████████████▊                       | 35/78 [01:09<01:08,  1.59s/it]INFO: [2024-06-25 12:58:09] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=KPNA2 with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:58:09] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:58:09] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:58:09] indra_db_rest.request_logs -   OFFSET: 0
 46%|███████████████████▍                   

INFO: [2024-06-25 12:58:36] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:58:36] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:58:36] indra_db_rest.request_logs -   OFFSET: 0
 67%|████████████████████████████              | 52/78 [01:38<00:49,  1.92s/it]INFO: [2024-06-25 12:58:38] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=CHAF1A with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:58:38] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:58:38] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:58:38] indra_db_rest.request_logs -   OFFSET: 0
 68%|████████████████████████████▌             | 53/78 [01:40<00:54,  2.20s/it]INFO: [2024-06-25 12:58:41] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=DTL with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:58:41] indra_db_rest.reque

INFO: [2024-06-25 12:59:04] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:59:04] indra_db_rest.request_logs -   OFFSET: 0
 88%|█████████████████████████████████████▏    | 69/78 [02:04<00:11,  1.23s/it]INFO: [2024-06-25 12:59:05] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=NUDT5 with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:59:05] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:59:05] indra_db_rest.request_logs -   LIMIT: None
INFO: [2024-06-25 12:59:05] indra_db_rest.request_logs -   OFFSET: 0
 90%|█████████████████████████████████████▋    | 70/78 [02:05<00:09,  1.13s/it]INFO: [2024-06-25 12:59:06] indra_db_rest.query_processor - Retrieving statements that have an agent where NAME=ORC1 with role=OBJECT and have type Phosphorylation.
INFO: [2024-06-25 12:59:06] indra_db_rest.request_logs - Running 0th request for statements
INFO: [2024-06-25 12:59:06] indra_db_rest.reque

In [19]:
# Now construct a dictionary of statements organized by specific sites making
# sure that phosphorylation of that specific site is described in the list
# of statements as values
stmts_by_site = {}
for site in valid_sites:
    stmts = stmts_by_target[site.gene_name]
    stmts = [s for s in stmts if s.enz and 'HGNC' in s.enz.db_refs]
    stmts = [s for s in stmts
             if s.residue == site.orig_res and s.position == site.orig_pos]
    stmts_by_site[(site.gene_name, site.orig_res, site.orig_pos)] = stmts
stmts_by_site

{('TICRR', 'S', '599'): [],
 ('DBF4', 'T', '345'): [],
 ('ATR', 'T', '1989'): [Phosphorylation(ATR(), ATR(), T, 1989),
  Phosphorylation(KDM5A(), ATR(), T, 1989),
  Phosphorylation(IVNS1ABP(), ATR(), T, 1989),
  Phosphorylation(SSB(), ATR(), T, 1989),
  Phosphorylation(KDM5B(), ATR(), T, 1989),
  Phosphorylation(PRPF19(), ATR(), T, 1989)],
 ('UFD1', 'S', '299'): [],
 ('UBE2T', 'S', '184'): [],
 ('XRCC6', 'S', '2'): [],
 ('EXO1', 'S', '714'): [Phosphorylation(ATM(), EXO1(), S, 714),
  Phosphorylation(ATR(), EXO1(), S, 714)],
 ('RAD51AP1', 'S', '19'): [],
 ('POLR2C', 'S', '124'): [],
 ('CDC25B', 'S', '321'): [Phosphorylation(CDK1(), CDC25B(), S, 321),
  Phosphorylation(MELK(), CDC25B(), S, 321),
  Phosphorylation(PRKCA(), CDC25B(), S, 321),
  Phosphorylation(CDC25B(), CDC25B(), S, 321),
  Phosphorylation(PRKACA(), CDC25B(), S, 321)],
 ('BAZ1B', 'S', '330'): [],
 ('MDC1', 'T', '455'): [Phosphorylation(CSNK2A1(), MDC1(), T, 455)],
 ('RIF1', 'S', '2205'): [Phosphorylation(CDK1(), RIF1(), S,

In [20]:
# Print statistics of the number of sites and the number with
# any known annotations
len(stmts_by_site), len([k for k, v in stmts_by_site.items() if v])

(174, 42)

In [21]:
# Explore specific examples of site annotations
[s for s in stmts_by_site.items() if s[0][0] == 'EXO1']

[(('EXO1', 'S', '714'),
  [Phosphorylation(ATM(), EXO1(), S, 714),
   Phosphorylation(ATR(), EXO1(), S, 714)]),
 (('EXO1', 'S', '610'), []),
 (('EXO1', 'T', '475'), []),
 (('EXO1', 'S', '598'), []),
 (('EXO1', 'S', '702'), []),
 (('EXO1', 'S', '639'), []),
 (('EXO1', 'S', '815'), []),
 (('EXO1', 'S', '700'), []),
 (('EXO1', 'S', '746'),
  [Phosphorylation(CHEK1(), EXO1(), S, 746),
   Phosphorylation(PRKAA1(), EXO1(), S, 746)])]

In [ ]:
protmapper.ann